# Analyzing Toronto Neighborhoods

#### by Sushant Deshpande

# --------Week 1--------

## SEGMENT 1 : Define a problem or an idea

A group of investors would like to open a restaurant in Toronto but aren't sure which one to open and where to open. In this project, we'll use python's web scraping to get the postal codes for Toronto’s neighborhoods, use the geographical coordinates provided to us in the last course and use FourSquare's API to get a list of nearby restaurants in a given locality.

We'll visualize the data using maps and bar charts. This will help us understand which type of restaurant is most popular in a given locality. With this information, investors can come to a decision.

## SEGMENT 2 : Describe the data

I started by getting the postal codes of Toronto from wikipedia using python's web scraping method. After that, I had to clean up the data and then merge the data with the geographical coordinate file provided to us in this course. Once I had all the postal codes with their corresponding geographical coordinates, the I leveraged the power of FourSquare to get a wealth of information.

FourSquare allowed me to get all the venues in my selected neighborhood. In this project, I only focused on East and West Toronto, but with FourSquare, the possibilities are unlimited. In the end, I had a clear picture of what is popular in which locality. With this knowledge, and investor and easily and quickly come to a decision as to where he wants to invest in in what.

# --------Week 2--------

### Still working on it...

In [ ]:
from bs4 import BeautifulSoup
from collections import defaultdict
import requests
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
%matplotlib inline
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
title = soup.title
text = soup.get_text()
rows = soup.find_all('tr')
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

df = pd.DataFrame(list_rows)
df.head(10)

In [ ]:
df.columns = ['title']
data1 = df['title'].str.split(',', expand=True)
data1.drop(data1.columns[[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]], axis=1, inplace=True)
data1.columns = ['PostalCode', 'Borough', 'Neighborhood']
data1.replace('\n','', regex=True)
data2 = data1.replace('\n]','', regex=True)
data2a = data1.PostalCode.str.replace("[", "")
data2.columns = ['PostalCode_1', 'Borough', 'Neighborhood']
data3 = pd.concat([data2a, data2], axis=1, sort=False)
data4 = data3.drop(['PostalCode_1'], axis=1)
data5 = data4.replace('Not assigned',np.nan, regex=True)
data6 = data5[pd.notnull(data5["Borough"])]
data6 = data6.reset_index(drop=True)
data6.head()

In [ ]:
data7a = data6['Neighborhood'].fillna(data6['Borough'])
data7 = pd.DataFrame(data7a)
data6.columns = ['PostalCode', 'Borough', 'Neighborhood_1']
data8a = pd.concat([data6, data7], axis=1, sort=False)
data8b = data8a.drop(['Neighborhood_1'], axis=1)
data8 = data8b[:212]
data9a = data8.groupby('PostalCode')['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
data9b = data8.groupby('PostalCode')['Borough'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
data9c = data9b['Borough'].str.split(', ').apply(set).str.join(', ')
data9d = pd.DataFrame(data9c)
data9d.head()
data9b.columns = ['PostalCode', 'Borough_1']
data9e = pd.concat([data9b, data9d], axis=1, sort=False)
data9f = data9e.drop(['Borough_1'], axis=1)
data9a.columns = ['PostalCode_1', 'Neighborhood']
data9g = pd.concat([data9f, data9a], axis=1, sort=False)
data9 = data9g.drop(['PostalCode_1'], axis=1)
data9.head()

In [ ]:
cor_df = pd.read_csv("Geospatial_Coordinates.csv")
cor_df.columns = ['PostalCode', 'Latitude', 'Longitude']
data10 = pd.merge(data9, cor_df, on=['PostalCode'])
data10[['Latitude', 'Longitude']] = data10[['Latitude', 'Longitude']].astype(float)
data10

In [ ]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of', address, 'are {}, {}.'.format(latitude, longitude))

In [ ]:
data_west = data10[data10['Borough'].str.contains('West Toronto')].reset_index(drop=True)
data_west

In [ ]:
data_east = data10[data10['Borough'].str.contains('East Toronto')].reset_index(drop=True)
data_east

In [ ]:
# create map of Toronto using latitude and longitude values
map_west = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_west['Latitude'],
                                           data_west['Longitude'],
                                           data_west['Borough'],
                                           data_west['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_west)  
    
map_west

In [ ]:
# create map of Toronto using latitude and longitude values
map_east = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_east['Latitude'],
                                           data_east['Longitude'],
                                           data_east['Borough'],
                                           data_east['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_east)  
    
map_east

In [ ]:
# @hidden_cell
CLIENT_ID = 'hidden' # your Foursquare ID
CLIENT_SECRET = 'hidden' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
data_east.loc[0, 'Neighborhood']

In [ ]:
data_west.loc[0, 'Neighborhood']

In [ ]:
neighborhood_latitude_east = data_east.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude_east = data_east.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name_east = data_east.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name_east, 
                                                               neighborhood_latitude_east, 
                                                               neighborhood_longitude_east))

In [ ]:
neighborhood_latitude_west = data_west.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude_west = data_west.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name_west = data_west.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name_west, 
                                                               neighborhood_latitude_west, 
                                                               neighborhood_longitude_west))

In [ ]:
LIMIT = 100
radius = 1000

url_4s_east = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_east, 
    neighborhood_longitude_east, 
    radius, 
    LIMIT)
url_4s_east

In [ ]:
LIMIT = 100
radius = 1000

url_4s_west = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude_west, 
    neighborhood_longitude_west, 
    radius, 
    LIMIT)
url_4s_west

In [ ]:
results_east = requests.get(url_4s_east).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues_east = results_east['response']['groups'][0]['items']
nearby_venues_east = json_normalize(venues_east) # flatten JSON
# filter columns
filtered_columns_east = ['venue.name',
                    'venue.categories',
                    'venue.location.lat',
                    'venue.location.lng']
nearby_venues_east =nearby_venues_east.loc[:, filtered_columns_east]
# filter the category for each row
nearby_venues_east['venue.categories'] = nearby_venues_east.apply(get_category_type, axis=1)
# clean columns
nearby_venues_east.columns = [col.split(".")[-1] for col in nearby_venues_east.columns]
nearby_venues_east

In [ ]:
results_west = requests.get(url_4s_west).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues_west = results_west['response']['groups'][0]['items']
nearby_venues_west = json_normalize(venues_west) # flatten JSON
# filter columns
filtered_columns_west = ['venue.name',
                    'venue.categories',
                    'venue.location.lat',
                    'venue.location.lng']
nearby_venues_west =nearby_venues_west.loc[:, filtered_columns_west]
# filter the category for each row
nearby_venues_west['venue.categories'] = nearby_venues_west.apply(get_category_type, axis=1)
# clean columns
nearby_venues_west.columns = [col.split(".")[-1] for col in nearby_venues_west.columns]
nearby_venues_west

In [ ]:
nearby_venues_west.shape

In [ ]:
print('East Toronto has {} venus,'.format(nearby_venues_east.shape[0]), 'and West Toronto has {} venus.'.format(nearby_venues_west.shape[0]))

In [ ]:
def getNearbyVenues_east(names, latitudes, longitudes, radius=1000):
    
    venues_list_east=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url_4s1_east = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results_east = requests.get(url_4s1_east).json()["response"]['groups'][0]['items']

        venues_list_east.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_east])

    nearby_venues_east = pd.DataFrame([item for venues_list_east in venues_list_east for item in venues_list_east])
    nearby_venues_east.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_east)

In [ ]:
def getNearbyVenues_west(names, latitudes, longitudes, radius=1000):
    
    venues_list_west=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url_4s1_west = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results_west = requests.get(url_4s1_west).json()["response"]['groups'][0]['items']

        venues_list_west.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_west])

    nearby_venues_west = pd.DataFrame([item for venues_list_west in venues_list_west for item in venues_list_west])
    nearby_venues_west.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_west)

In [ ]:
toronto_venues_east = getNearbyVenues_east(names=data_east['Neighborhood'],
                                   latitudes=data_east['Latitude'],
                                   longitudes=data_east['Longitude']
                                  )

In [ ]:
toronto_venues_west = getNearbyVenues_west(names=data_west['Neighborhood'],
                                   latitudes=data_west['Latitude'],
                                   longitudes=data_west['Longitude']
                                  )

In [ ]:
toronto_venues_east

In [ ]:
toronto_venues_west

In [ ]:
toronto_venues_east.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues_east['Venue Category'].unique())))

In [ ]:
toronto_venues_west.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues_west['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot_east = pd.get_dummies(toronto_venues_east[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot_east['Neighborhood'] = toronto_venues_east['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns_east = [toronto_onehot_east.columns[-1]] + list(toronto_onehot_east.columns[:-1])
toronto_onehot_east = toronto_onehot_east[fixed_columns_east]

toronto_onehot_east

In [ ]:
# one hot encoding
toronto_onehot_west = pd.get_dummies(toronto_venues_west[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot_west['Neighborhood'] = toronto_venues_west['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns_west = [toronto_onehot_west.columns[-1]] + list(toronto_onehot_west.columns[:-1])
toronto_onehot_west = toronto_onehot_west[fixed_columns_west]

toronto_onehot_west

In [ ]:
toronto_grouped_east = toronto_onehot_east.groupby('Neighborhood').mean().reset_index()
num_top_venues_east = 5

for hood in toronto_grouped_east['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped_east[toronto_grouped_east['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues_east))
    print('\n')

In [ ]:
toronto_grouped_west = toronto_onehot_west.groupby('Neighborhood').mean().reset_index()
num_top_venues_west = 5

for hood in toronto_grouped_west['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped_west[toronto_grouped_west['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues_west))
    print('\n')

In [ ]:
def return_most_common_venues_east(row, num_top_venues_east):
    row_categories_east = row.iloc[1:]
    row_categories_sorted_east = row_categories_east.sort_values(ascending=False)
    
    return row_categories_sorted_east.index.values[0:num_top_venues_east]

In [ ]:
def return_most_common_venues_west(row, num_top_venues_west):
    row_categories_west = row.iloc[1:]
    row_categories_sorted_west = row_categories_west.sort_values(ascending=False)
    
    return row_categories_sorted_west.index.values[0:num_top_venues_west]

In [ ]:
num_top_venues_east = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues_east):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_east = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_east['Neighborhood'] = toronto_grouped_east['Neighborhood']

for ind in np.arange(toronto_grouped_east.shape[0]):
    neighborhoods_venues_sorted_east.iloc[ind, 1:] = return_most_common_venues_east(toronto_grouped_east.iloc[ind, :], num_top_venues_east)

neighborhoods_venues_sorted_east

In [ ]:
num_top_venues_west = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues_west):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_west = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_west['Neighborhood'] = toronto_grouped_west['Neighborhood']

for ind in np.arange(toronto_grouped_west.shape[0]):
    neighborhoods_venues_sorted_west.iloc[ind, 1:] = return_most_common_venues_west(toronto_grouped_west.iloc[ind, :], num_top_venues_west)

neighborhoods_venues_sorted_west

In [ ]:
kclusters = 5

toronto_grouped_clustering_east = toronto_grouped_east.drop('Neighborhood', 1)

kmeans_east = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering_east)

kmeans_east.labels_[0:10]

In [ ]:
kclusters = 5

toronto_grouped_clustering_west = toronto_grouped_west.drop('Neighborhood', 1)

kmeans_west = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering_west)

kmeans_west.labels_[0:10]

In [ ]:
toronto_merged_east = data_east

toronto_merged_east['Cluster Labels'] = kmeans_east.labels_

toronto_merged_east = toronto_merged_east.join(neighborhoods_venues_sorted_east.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_east.head()

In [ ]:
toronto_merged_west = data_west

toronto_merged_west['Cluster Labels'] = kmeans_west.labels_

toronto_merged_west = toronto_merged_west.join(neighborhoods_venues_sorted_west.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_west.head()

In [ ]:
map_clusters_east = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_east['Latitude'],
                                  toronto_merged_east['Longitude'],
                                  toronto_merged_east['Neighborhood'],
                                  toronto_merged_east['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_east)
       
map_clusters_east

In [ ]:
map_clusters_west = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_west['Latitude'],
                                  toronto_merged_west['Longitude'],
                                  toronto_merged_west['Neighborhood'],
                                  toronto_merged_west['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_west)
       
map_clusters_west

In [ ]:
toronto_east_group = toronto_venues_east.groupby('Venue Category').count()
print('There are {} uniques categories.'.format(len(toronto_venues_east['Venue Category'].unique())))

In [ ]:
toronto_west_group = toronto_venues_west.groupby('Venue Category').count()
print('There are {} uniques categories.'.format(len(toronto_venues_west['Venue Category'].unique())))

In [ ]:
toronto_east_group.sort_values(by='Neighborhood', ascending=True, inplace=True)

df_top10_east = toronto_east_group['Neighborhood'].tail(10)
df_top10_east

In [ ]:
toronto_west_group.sort_values(by='Neighborhood', ascending=True, inplace=True)

df_top10_west = toronto_west_group['Neighborhood'].tail(10)
df_top10_west

In [ ]:
df_top10_east.plot(kind='barh', figsize=(12, 12), color='limegreen')
plt.xlabel('Kind of Shops')
plt.title('Top 15 kind of shops in Totonto - East')

plt.show()

In [ ]:
df_top10_west.plot(kind='barh', figsize=(12, 12), color='orange')
plt.xlabel('Kind of Shops')
plt.title('Top 15 kind of shops in Totonto - West')

plt.show()